In [1]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


/home/gabrielpitali/Documents/tg-eng-info


In [32]:
import pandas as pd
import numpy as np
import json

if 1 == 0:
  faces_path = 'src/result_faces_scaleFactor_13_minNeighbors_5_minSize_30_30.json'
  no_faces_path = 'src/result_no_faces_scaleFactor_13_minNeighbors_5_minSize_30_30.json'
  test_check = 1
else:
  faces_path = 'src/result_faces_scaleFactor_105_minNeighbors_3_minSize_30_30.json'
  no_faces_path = 'src/result_no_faces_scaleFactor_105_minNeighbors_3_minSize_30_30.json'
  test_check = 1

with open(faces_path) as json_file:
  result_faces = json.load(json_file)
result_faces["raw_result"] = result_faces["raw_result"][:17130]
#print(len(result_faces["raw_result"]))

with open(no_faces_path) as json_file:
  result_no_faces = json.load(json_file)
result_no_faces["raw_result"] = result_no_faces["raw_result"][:17130]
#print(len(result_no_faces["raw_result"]))


In [33]:
# Count faces

def add_counts(result_data):
  result_data["count_imgs"] = len(result_data["raw_result"])
  result_data["count_0"] = 0
  result_data["count_1"] = 0
  result_data["count_2"] = 0
  result_data["count_1+"] = 0
  for img in result_data["raw_result"]:
      if img["face_count"] == 0:
        result_data["count_0"] += 1
      elif img["face_count"] == 1:
        result_data["count_1"] += 1
        result_data["count_1+"] += 1
      else:
        result_data["count_2"] += 1
        result_data["count_1+"] += 1
  print(str(result_data["count_0"]) + " images with 0 faces (" + str(result_data["count_0"] * 100 / result_data["count_imgs"]) + "%)")
  print(str(result_data["count_1"]) + " images with 1 faces (" + str(result_data["count_1"]*100/result_data["count_imgs"]) + "%)")
  print(str(result_data["count_1+"]) + " images with 1 or more faces (" + str(result_data["count_1+"]*100/result_data["count_imgs"]) + "%)")
  print(str(result_data["count_2"]) + " images with 2 or more faces (" + str(result_data["count_2"]*100/result_data["count_imgs"]) + "%)")

print("=== Faces Dataset ===")
add_counts(result_faces)

print("=== No Faces Dataset ===")
add_counts(result_no_faces)


=== Faces Dataset ===
283 images with 0 faces (1.6520723876240513%)
8789 images with 1 faces (51.30764740221833%)
16847 images with 1 or more faces (98.34792761237595%)
8058 images with 2 or more faces (47.04028021015762%)
=== No Faces Dataset ===
11373 images with 0 faces (66.3922942206655%)
4269 images with 1 faces (24.92119089316988%)
5757 images with 1 or more faces (33.6077057793345%)
1488 images with 2 or more faces (8.686514886164623%)


In [34]:
from sklearn.metrics import confusion_matrix

y_true = (["face"] * result_faces["count_imgs"]) + (["nao_face"] * result_no_faces["count_imgs"])
y_pred = (["face"] * result_faces["count_1+"]) + (["nao_face"] * result_faces["count_0"]) + (["face"] * result_no_faces["count_1+"]) + (["nao_face"] * result_no_faces["count_0"])

assert len(y_true) == len(y_pred), "Mesmo número de items existentes e previstos"

matrix = confusion_matrix(y_true, y_pred, labels=["face", "nao_face"])

tp, fn, fp, tn = [i/len(y_true) for i in matrix.ravel()]

result_print = np.array([[tp, fn, (tp+fn)],
                         [fp, tn, (fp+tn)],
                         [(tp+fp), (fn+tn), (tp*fn*fp*tn)]]) *100

print("sensibilidade")
print(tp / (tp + fn))
print("especificidade")
print(tn / (tn + fp))
pd.DataFrame(result_print, columns=["(B) Face (previsto)", "(xB) Não Face (previsto)", ""], index=["(A) Face (real)", "(xA) Não Face (real)", ""])


sensibilidade
0.9834792761237595
especificidade
0.663922942206655


,(B) Face (previsto),(xB) Não Face (previsto),
(A) Face (real),49.173964,0.826036,50.000000
(xA) Não Face (real),16.803853,33.196147,50.000000
,65.977817,34.022183,0.022658


In [24]:

h = 0.5 # Manual analysis cost for each picture
v = 250.37 # Approved customer value

g0 = (tp+fn)*v - (tp+fp+fn+tn)*h
g1 = tp*v - (tp+fp)*h

g1 = tp*v - (tp+fp)*h

#for i in range(1,5):
#  g1 += (0.5**i)*fn*(tp*v - (tp+fp)*h)

print(g1 - g0)

#print((fn+tn)/fn)
#print(v/h)

#print(h_cost*(fn+tn) - fn*c_value)


-9.697114100209347


# This is a test cell

To see how git handle the diff